# Modelo de previsão se um cliente deixará o banco em breve.

# Contents <a id='back'></a>

* [Introdução](#intro)
* [Etapa 1. Visão geral dos dados](#data_review)
* [Etapa 2. Transformações dos dados](#tranforamation_data)
* [Etapa 3. Modelo](#model)
* [Conclusões](#end)

## Introdução <a id='intro'></a>

Os clientes do Beta Bank estão saindo: pouco a pouco, escapulindo todo mês. Os banqueiros descobriram que é mais barato manter os clientes existentes do que atrair novos.

Precisamos prever se um cliente vai deixar o banco em breve. Você tem os dados sobre o comportamento passado dos clientes e rescisões de contratos com o banco.

Construa um modelo com o valor máximo possível de F1. Para passar na revisão, você precisa de um F1-score de pelo menos 0,59 para o conjunto de dados de teste. 

Além disso, meça a métrica AUC-ROC e compare-a com o F1-score.

### Objetivo: 
- Como você preparou os dados para o treinamento? Você processou todos os tipos de características?
- Você explicou bem os passos de pré-processamento?
- Como você investigou o equilíbrio das classes?
- Você estruturou o modelo sem levar em conta o desequilíbrio de classes?
- Quais são suas descobertas relacionadas à tarefa em questão?
- Você dividiu corretamente os dados em conjuntos?
- Como você tem trabalhado com o desequilíbrio de classes?
- Você usou pelo menos duas técnicas para correção de desequilíbrio?
- Você executou o treinamento, validação e teste final do modelo corretamente?
- Qual é o seu F1-score?
- Você examinou os valores de AUC-ROC?




### Descrição de dados

- `Abra e examine o arquivo de dados. Caminho para o arquivo:`
/datasets/Churn.csv. Baixar o conjunto de dados

[Voltar ao Índice](#back)

## Etapa 1. Visão geral dos dados <a id='data_review'></a>

Carregue os dados das consultas, explore-os e carregue bibliotecas que você acredita que são necessárias para o projeto.

In [1]:
# Bibliotecas para manipulação e análise de dados
import pandas as pd  # Pandas é utilizado para manipulação de dados e estruturas de dados
import numpy as np   # NumPy é utilizado para operações numéricas e manipulação de arrays

# Bibliotecas para visualização de dados
import matplotlib.pyplot as plt  # Matplotlib é utilizado para criar gráficos e visualizações

# Bibliotecas para estatísticas e testes
from scipy import stats as st  # SciPy oferece funções estatísticas e de teste

# Bibliotecas para aprendizado de máquina
from sklearn.model_selection import train_test_split  # Para dividir os dados em conjuntos de treino e teste
from sklearn.tree import DecisionTreeClassifier  # Classificador de árvore de decisão
from sklearn.ensemble import RandomForestClassifier  # Classificador de floresta aleatória
from sklearn.linear_model import LogisticRegression  # Regressão logística

# Bibliotecas para avaliação de modelos
from sklearn.metrics import accuracy_score  # Para calcular a acurácia do modelo
from sklearn.metrics import confusion_matrix  # Para criar a matriz de confusão
from sklearn.metrics import recall_score  # Para calcular a sensibilidade (recall)
from sklearn.metrics import precision_score  # Para calcular a precisão
from sklearn.metrics import roc_auc_score  # Para calcular a área sob a curva ROC

# Biblioteca para codificação de variáveis categóricas
from sklearn.preprocessing import OrdinalEncoder  # Para codificar variáveis categóricas em números

## Carregue os Dados

In [2]:
# Carregue o arquivo com os dados em um DataFrame
df = pd.read_csv('C:/Users/gabri/Downloads/Corrigido/Projeto 8/Churn.csv')

In [3]:
# Vamos ver quantas linhas e colunas nosso conjunto de dados tem
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
# vamos exibir as primeiras 10 linhas
df.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


## Explore os dados iniciais

**Descrição dos dados**
- `RowNumber` - (índice das strings de dados)
- `CustomerId` - (identificador exclusivo do cliente)
- `Surname` - (sobrenome)
- `CreditScore` - (pontuação de crédito)
- `Geography` - (país de residência)
- `Gender` - (gênero)
- `Age` - (idade)
- `Tenure` - (tempo de serviço para o cliente)
- `Balance` - (saldo da conta)
- `NumOfProducts` - (número de produtos bancários usados pelo cliente)
- `HasCrCard` - (cliente possui cartão de crédito (1 - sim; 0 - não))
- `IsActiveMember` - (cliente ativo (1 - sim; 0 - não))
- `EstimatedSalary` - (salário estimado `сalls` - (número de chamadas))
- `Exited` - (o cliente saiu (1 - sim; 0 - não))

[Voltar ao Índice](#back)


### Duplicatas <a id='duplicates'></a>
Encontrando de duplicatas óbvias na tabela:

In [5]:
# Verificar Valores Duplicados
df.duplicated().sum()

0

### Valores ausentes <a id='missing_values'></a>

In [6]:
# Verificar Valores Ausentes
df.isna().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [7]:
(df.isnull().sum() / df.shape[0]) * 100

RowNumber          0.00
CustomerId         0.00
Surname            0.00
CreditScore        0.00
Geography          0.00
Gender             0.00
Age                0.00
Tenure             9.09
Balance            0.00
NumOfProducts      0.00
HasCrCard          0.00
IsActiveMember     0.00
EstimatedSalary    0.00
Exited             0.00
dtype: float64

Podemos observar que 9.09% dos nossos dados na coluna 'Tenure' estão faltantes, e não há duplicatas.Mas o nome das colunas não estão de acordo com as regras da boa prática de estilo, pois isso, iremos renomear esses nomes e verificar se há dados inconsistentes.

[Voltar ao Índice](#back)

## Etapa 2. Transformação de dados <a id='tranforamation_data'></a>
Vamos examinar cada coluna para ver quais problemas podemos ter nelas.

In [8]:
df['CustomerId'].sort_values()

1287    15565701
4198    15565706
7090    15565714
2020    15565779
3697    15565796
          ...   
3411    15815628
8271    15815645
8088    15815656
1762    15815660
5502    15815690
Name: CustomerId, Length: 10000, dtype: int64

In [9]:
df['Surname'].sort_values()

3271     Abazu
5109     Abazu
841      Abbie
2988    Abbott
4573    Abbott
         ...  
1030      Zuev
586      Zuyev
6973     Zuyev
6385    Zuyeva
7739    Zuyeva
Name: Surname, Length: 10000, dtype: object

In [10]:
df['CreditScore'].describe()

count    10000.000000
mean       650.528800
std         96.653299
min        350.000000
25%        584.000000
50%        652.000000
75%        718.000000
max        850.000000
Name: CreditScore, dtype: float64

In [11]:
df['Geography'].sort_values().unique()

array(['France', 'Germany', 'Spain'], dtype=object)

In [12]:
df['Gender'].sort_values().unique()

array(['Female', 'Male'], dtype=object)

In [13]:
df['Age'].describe()

count    10000.000000
mean        38.921800
std         10.487806
min         18.000000
25%         32.000000
50%         37.000000
75%         44.000000
max         92.000000
Name: Age, dtype: float64

In [14]:
df['Tenure'].sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., nan])

In [15]:
df['Balance'].describe()

count     10000.000000
mean      76485.889288
std       62397.405202
min           0.000000
25%           0.000000
50%       97198.540000
75%      127644.240000
max      250898.090000
Name: Balance, dtype: float64

In [16]:
df['NumOfProducts'].sort_values().unique()

array([1, 2, 3, 4], dtype=int64)

In [17]:
df['HasCrCard'].sort_values().unique()

array([0, 1], dtype=int64)

In [18]:
df['IsActiveMember'].sort_values().unique()

array([0, 1], dtype=int64)

In [19]:
df['EstimatedSalary'].describe()

count     10000.000000
mean     100090.239881
std       57510.492818
min          11.580000
25%       51002.110000
50%      100193.915000
75%      149388.247500
max      199992.480000
Name: EstimatedSalary, dtype: float64

In [20]:
df['Exited'].sort_values().unique()

array([0, 1], dtype=int64)

### Conclusões intermediárias <a id='data_preprocessing_conclusions_intermediary'></a>

Não há valores deproporcionais, nessa questão está tudo coerente.

[Voltar ao Índice](#back)


## Etapa 3. Transformação de dados <a id='tranforamation_data'>

In [21]:
# a lista dos nomes das colunas na tabela df
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [22]:
# renomeando colunas
df = df.rename(str.lower, axis='columns')

In [23]:
# a lista dos nomes das colunas na tabela df
df.columns

Index(['rownumber', 'customerid', 'surname', 'creditscore', 'geography',
       'gender', 'age', 'tenure', 'balance', 'numofproducts', 'hascrcard',
       'isactivemember', 'estimatedsalary', 'exited'],
      dtype='object')

### Preenchendo valores ausentes na coluna 'tenure'

In [24]:
# Vejamos a tabela filtrada com valores ausentes na primeira coluna com dados ausentes
df[df['tenure'].isna()].head(50)

,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
30,31,15589475,Azikiwe,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,49,15766205,Yin,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,52,15768193,Trevisani,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,54,15702298,Parkhill,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,61,15651280,Hunter,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
82,83,15641732,Mills,543,France,Female,36,NaN,0.00,2,0,0,26019.59,0
85,86,15805254,Ndukaku,652,Spain,Female,75,NaN,0.00,2,1,1,114675.75,0
94,95,15676966,Capon,730,Spain,Male,42,NaN,0.00,2,0,1,85982.47,0
99,100,15633059,Fanucci,413,France,Male,34,NaN,0.00,2,0,0,6534.18,0
111,112,15665790,Rowntree,538,Germany,Male,39,NaN,108055.10,2,1,0,27231.26,0


Não há indicios que podemos substituir esses valores faltantes dessa coluna com base nas outras,segue muito aleatorio, por isso substituiremos com valor de 0.

In [25]:
#Substituindo valores faltantes vcom 0
df['tenure'].fillna(0, inplace = True)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rownumber        10000 non-null  int64  
 1   customerid       10000 non-null  int64  
 2   surname          10000 non-null  object 
 3   creditscore      10000 non-null  int64  
 4   geography        10000 non-null  object 
 5   gender           10000 non-null  object 
 6   age              10000 non-null  int64  
 7   tenure           10000 non-null  float64
 8   balance          10000 non-null  float64
 9   numofproducts    10000 non-null  int64  
 10  hascrcard        10000 non-null  int64  
 11  isactivemember   10000 non-null  int64  
 12  estimatedsalary  10000 non-null  float64
 13  exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Concluimos os pré-processamento de dados, não há mais valores ausentes, duplicados ou tipo de dados errado.

[Voltar ao Índice](#back)

## Etapa 3. Modelo <a id='model'></a>

In [27]:
#Transformando Variaveis categoricas em variáveis numéricas
df_gender = pd.get_dummies(df['gender']).reset_index()
df_geography = pd.get_dummies(df['geography']).reset_index()

In [28]:
# Realiza a junção (merge) dos DataFrames df_gender e df_geography
# Utiliza a junção do tipo 'left', o que significa que todas as linhas do df_gender serão mantidas
# e as correspondências do df_geography serão adicionadas onde houver correspondência na coluna 'index'
df_index = pd.merge(df_gender, df_geography, how='left', on='index')

In [29]:
df_index

,index,Female,Male,France,Germany,Spain
0,0,True,False,True,False,False
1,1,True,False,False,False,True
2,2,True,False,True,False,False
3,3,True,False,True,False,False
4,4,True,False,False,False,True
...,...,...,...,...,...,...
9995,9995,False,True,True,False,False
9996,9996,False,True,True,False,False
9997,9997,True,False,True,False,False
9998,9998,False,True,False,True,False


In [30]:
# Remove as colunas 'gender' e 'geography' do DataFrame df
# O parâmetro axis=1 indica que as colunas estão sendo removidas (em oposição a linhas)
# Após a remoção, o índice do DataFrame é redefinido com reset_index()
df_ohe = df.drop(['gender', 'geography'], axis=1).reset_index()

In [31]:
df_final = pd.merge(df_ohe, df_index, how = 'left', on = 'index')

In [32]:
df_final

,index,rownumber,customerid,surname,creditscore,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited,Female,Male,France,Germany,Spain
0,0,1,15634602,Hargrave,619,42,2.0,0.00,1,1,1,101348.88,1,True,False,True,False,False
1,1,2,15647311,Hill,608,41,1.0,83807.86,1,0,1,112542.58,0,True,False,False,False,True
2,2,3,15619304,Onio,502,42,8.0,159660.80,3,1,0,113931.57,1,True,False,True,False,False
3,3,4,15701354,Boni,699,39,1.0,0.00,2,0,0,93826.63,0,True,False,True,False,False
4,4,5,15737888,Mitchell,850,43,2.0,125510.82,1,1,1,79084.10,0,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9996,15606229,Obijiaku,771,39,5.0,0.00,2,1,0,96270.64,0,False,True,True,False,False
9996,9996,9997,15569892,Johnstone,516,35,10.0,57369.61,1,1,1,101699.77,0,False,True,True,False,False
9997,9997,9998,15584532,Liu,709,36,7.0,0.00,1,0,1,42085.58,1,True,False,True,False,False
9998,9998,9999,15682355,Sabbatini,772,42,3.0,75075.31,2,1,0,92888.52,1,False,True,False,True,False


In [33]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   index            10000 non-null  int64  
 1   rownumber        10000 non-null  int64  
 2   customerid       10000 non-null  int64  
 3   surname          10000 non-null  object 
 4   creditscore      10000 non-null  int64  
 5   age              10000 non-null  int64  
 6   tenure           10000 non-null  float64
 7   balance          10000 non-null  float64
 8   numofproducts    10000 non-null  int64  
 9   hascrcard        10000 non-null  int64  
 10  isactivemember   10000 non-null  int64  
 11  estimatedsalary  10000 non-null  float64
 12  exited           10000 non-null  int64  
 13  Female           10000 non-null  bool   
 14  Male             10000 non-null  bool   
 15  France           10000 non-null  bool   
 16  Germany          10000 non-null  bool   
 17  Spain        

In [34]:
# Divide o DataFrame df_final em dois conjuntos: df_train_valid e df_test
# 20% dos dados são reservados para o conjunto de teste (df_test)
# random_state=12345 garante que a divisão seja reprodutível
# stratify=df_final['exited'] assegura que a proporção da variável 'exited' seja mantida em ambos os conjuntos
df_train_valid, df_test = train_test_split(df_final, test_size=0.2, random_state=12345, stratify=df_final['exited'])

# Divide o DataFrame df_train_valid em dois conjuntos: df_train e df_valid
# 25% dos dados de df_train_valid são reservados para o conjunto de validação (df_valid)
# Isso resulta em 60% dos dados originais para treino e 20% para validação
# random_state=12345 garante que a divisão seja reprodutível
# stratify=df_train_valid['exited'] assegura que a proporção da variável 'exited' seja mantida em ambos os conjuntos
df_train, df_valid = train_test_split(df_train_valid, test_size=0.25, random_state=12345, stratify=df_train_valid['exited'])


In [35]:
# Remove colunas desnecessárias do DataFrame df_train para criar o conjunto de características (features)
# As colunas removidas são 'index', 'surname', 'rownumber', 'customerid' e 'exited'
# O resultado é armazenado em features_train
features_train = df_train.drop(['index', 'surname', 'rownumber', 'customerid', 'exited'], axis=1)

# Armazena a coluna 'exited' do DataFrame df_train como o alvo (target) para o modelo de treino
target_train = df_train['exited']

# Repete o processo para o conjunto de validação (df_valid)
# Remove colunas desnecessárias e armazena as características em features_valid
features_valid = df_valid.drop(['index', 'surname', 'rownumber', 'customerid', 'exited'], axis=1)

# Armazena a coluna 'exited' do DataFrame df_valid como o alvo (target) para o modelo de validação
target_valid = df_valid['exited']

# Repete o processo para o conjunto de teste (df_test)
# Remove colunas desnecessárias e armazena as características em features_test
features_test = df_test.drop(['index', 'surname', 'rownumber', 'customerid', 'exited'], axis=1)

# Armazena a coluna 'exited' do DataFrame df_test como o alvo (target) para o modelo de teste
target_test = df_test['exited']


## Melhorando o modelo

In [36]:
# Itera sobre diferentes valores de profundidade máxima para a árvore de decisão, variando de 1 a 13
for depth in range(1, 14):
    # Cria um classificador de árvore de decisão com um estado aleatório fixo e a profundidade máxima definida
    model_dctc = DecisionTreeClassifier(random_state=123456, max_depth=depth)
    
    # Treina o modelo usando as características e alvos do conjunto de treino
    model_dctc.fit(features_train, target_train)
    
    # Faz previsões no conjunto de validação usando o modelo treinado
    predictions_valid = model_dctc.predict(features_valid)
    
    # Exibe a profundidade máxima utilizada e a acurácia das previsões no conjunto de validação
    print('max_depth =', depth, ': ', end='')
    print(accuracy_score(target_valid, predictions_valid))


max_depth = 1 : 0.7965
max_depth = 2 : 0.836
max_depth = 3 : 0.835
max_depth = 4 : 0.8495
max_depth = 5 : 0.846
max_depth = 6 : 0.8535
max_depth = 7 : 0.8535
max_depth = 8 : 0.855
max_depth = 9 : 0.8485
max_depth = 10 : 0.8405
max_depth = 11 : 0.8315
max_depth = 12 : 0.8195
max_depth = 13 : 0.815


Fizemos um For para saber quanto max_depth está a melhor acurácia, e o melhor foi o 8, aplicaremos abaixo e procuraremos o melhor n_estimators pra o nosso modelo.

In [37]:
# Inicializa as variáveis para armazenar a melhor acurácia e o melhor número de estimadores
best_score = 0
best_est = 0

# Itera sobre diferentes valores de n_estimators para o modelo de floresta aleatória, variando de 1 a 10
for est in range(1, 11):
    # Cria um classificador de floresta aleatória com um estado aleatório fixo, profundidade máxima de 8 e n_estimators definido
    model_best = RandomForestClassifier(random_state=123456, max_depth=8, n_estimators=est)
    
    # Treina o modelo usando as características e alvos do conjunto de treino
    model_best.fit(features_train, target_train)
    
    # Avalia o modelo no conjunto de validação e armazena a acurácia
    score = model_best.score(features_valid, target_valid)
    
    # Se a acurácia atual for melhor que a melhor acurácia registrada, atualiza as variáveis
    if score > best_score:
        best_score = score
        best_est = est

# Exibe a acurácia do melhor modelo encontrado e o número de estimadores correspondente
print("A acurácia do melhor modelo no conjunto de validação (n_estimators = {}): {}".format(best_est, best_score))


A acurácia do melhor modelo no conjunto de validação (n_estimators = 10): 0.8575


n_estimators = 10 foi o queteve melhor acurácia

[Voltar ao Índice](#back)

## Valor de F1 sem Balanceamento de Classe

In [38]:
# Cria um classificador de floresta aleatória com um estado aleatório fixo, profundidade máxima de 8 e 10 estimadores
model = RandomForestClassifier(random_state=123456, max_depth=8, n_estimators=10)

# Treina o modelo usando as características e alvos do conjunto de treino
model.fit(features_train, target_train)

# Faz previsões no conjunto de validação usando o modelo treinado
model_valid = model.predict(features_valid)


In [39]:
# Calcula a acurácia das previsões no conjunto de validação
# Compara as previsões feitas pelo modelo (model_valid) com os valores reais (target_valid)
accuracy = accuracy_score(target_valid, model_valid)

# Exibe a acurácia calculada
print("Acurácia no conjunto de validação:", accuracy)


Acurácia no conjunto de validação: 0.8575


In [40]:
# Calcula a matriz de confusão para as previsões no conjunto de validação
# Compara as previsões feitas pelo modelo (model_valid) com os valores reais (target_valid)
conf_matrix = confusion_matrix(target_valid, model_valid)

# Exibe a matriz de confusão
print("Matriz de Confusão:\n", conf_matrix)


Matriz de Confusão:
 [[1556   37]
 [ 248  159]]


In [41]:
from sklearn.metrics import classification_report

In [42]:
print(classification_report(target_valid, model_valid))

              precision    recall  f1-score   support

           0       0.86      0.98      0.92      1593
           1       0.81      0.39      0.53       407

    accuracy                           0.86      2000
   macro avg       0.84      0.68      0.72      2000
weighted avg       0.85      0.86      0.84      2000



Como podemos analizar acima, há um desequilíbrio das classes, estudamos o modelo, e percebemos que existe um desbalancemaneto monstrado na Matrix de Confusão, usaremos duas técnicas para que não haja superestimativas do desempenho do modelo.

## Valor de F1 com Balanceamento de Classe

In [43]:
from sklearn.utils import shuffle

# Define a função upsample para aumentar a amostra da classe minoritária
def upsample(features, target, repeat):
    # Separa as características e alvos das classes 0 e 1
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    # Duplica as amostras da classe 1 (ou seja, a classe minoritária) 'repeat' vezes
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    # Embaralha os dados para garantir que a ordem não influencie o modelo
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    # Retorna as características e alvos aumentados
    return features_upsampled, target_upsampled

# Chama a função upsample para aumentar a amostra do conjunto de treino, repetindo a classe 1 três vezes
features_upsampled, target_upsampled = upsample(features_train, target_train, 3)


In [44]:
# Cria um classificador de floresta aleatória com um estado aleatório fixo, profundidade máxima de 8, 10 estimadores
# e ajusta os pesos das classes para lidar com o desbalanceamento
model_upsampled = RandomForestClassifier(random_state=123456, max_depth=8, n_estimators=10, class_weight='balanced')

# Treina o modelo usando as características e alvos aumentados
model_upsampled.fit(features_upsampled, target_upsampled)

# Faz previsões no conjunto de dados aumentados usando o modelo treinado
model_valid_upsampled = model_upsampled.predict(features_upsampled)


In [45]:
accuracy_score(target_upsampled, model_valid_upsampled)

0.8364906464598627

In [46]:
confusion_matrix(target_upsampled, model_valid_upsampled)

array([[4137,  640],
       [ 741, 2928]], dtype=int64)

In [47]:
print(classification_report(target_upsampled, model_valid_upsampled))

              precision    recall  f1-score   support

           0       0.85      0.87      0.86      4777
           1       0.82      0.80      0.81      3669

    accuracy                           0.84      8446
   macro avg       0.83      0.83      0.83      8446
weighted avg       0.84      0.84      0.84      8446



In [48]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.3)


In [49]:
# Cria um classificador de floresta aleatória com um estado aleatório fixo, profundidade máxima de 8, 10 estimadores
# e ajusta os pesos das classes para lidar com o desbalanceamento
model_downsampled = RandomForestClassifier(random_state=123456, max_depth=8, n_estimators=10, class_weight='balanced')

# Treina o modelo usando as características e alvos reduzidos
model_downsampled.fit(features_downsampled, target_downsampled)

# Faz previsões no conjunto de dados reduzidos usando o modelo treinado
model_valid_downsampled = model_downsampled.predict(features_downsampled)


In [50]:
accuracy_score(target_downsampled, model_valid_downsampled)

0.8445030120481928

In [51]:
confusion_matrix(target_downsampled, model_valid_downsampled)

array([[1254,  179],
       [ 234,  989]], dtype=int64)

In [52]:
print(classification_report(target_downsampled, model_valid_downsampled))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86      1433
           1       0.85      0.81      0.83      1223

    accuracy                           0.84      2656
   macro avg       0.84      0.84      0.84      2656
weighted avg       0.84      0.84      0.84      2656



Podemos notar que nosso modelo caiu um pouco de performace mas está melhor equilibrado, sendo a melhor performace 84% usando downsampled e class_weight.

Acurácia de 85% sem o balanceamento de classe, e 84% com o balanceamento de classe, podemos notar uma piora do nosso modelo, porém um modelo mais balanceado.

## Valor de AUC-ROC

In [53]:
# Obtém as probabilidades das classes para o conjunto de validação usando o modelo treinado
probabilities_valid = model_downsampled.predict_proba(features_valid)

# Extrai as probabilidades da classe 1 (ou seja, a classe positiva)
probabilities_one_valid = probabilities_valid[:, 1]

# Calcula a AUC-ROC (Área sob a Curva Receiver Operating Characteristic) usando as verdadeiras classes e as probabilidades da classe 1
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

# Exibe o valor da AUC-ROC calculada
print(auc_roc)


0.8564010852146445


Podemos ver que nosso AUC-ROC teve um valor bastante satisfatório

## Aplicando o modelo no conjunto de teste

In [54]:
# Cria um classificador de floresta aleatória com um estado aleatório fixo, profundidade máxima de 8, 10 estimadores
# e ajusta os pesos das classes para lidar com o desbalanceamento
final_model = RandomForestClassifier(random_state=123456, max_depth=8, n_estimators=10, class_weight='balanced')

# Treina o modelo usando as características e alvos do conjunto de validação
final_model.fit(features_valid, target_valid)

# Faz previsões no conjunto de teste usando o modelo treinado
predictions_test = final_model.predict(features_test)


In [55]:
accuracy_score(target_test,predictions_test)

0.8155

In [56]:
print(classification_report(target_test, predictions_test))

              precision    recall  f1-score   support

           0       0.89      0.88      0.88      1593
           1       0.54      0.57      0.56       407

    accuracy                           0.82      2000
   macro avg       0.72      0.72      0.72      2000
weighted avg       0.82      0.82      0.82      2000



Criamos um modelo com 81.5% de acurácia para a previsão se um cliente deixará o banco em breve.

## Conclusão geral <a id='end'></a>

Neste projeto, importamos bibliotecas como "pandas" e "numpy" que habitualmente já utilizamos, e sklearn para criação do nosso modelo.

Verificamos se realmente o pre-processamento foi feito, para que não tivesse acontecido algum problema na criação do nosso modelo.

E desenvolvemos um modelo para recomedação, e usamos a floresta aleatória pois sua acurácia é alta mediante a treinamento dos modelos,tivemos sucesso no valor de F1 e na distribuição de classes com dummies.

[Voltar ao Índice](#back)